# 04: Bounded Contexts — Multi-Instance Kind Definitions

The example codebase in `example-codebase/` is a single clean-architecture shop with one `locate<ShopContext>()` instance. What happens when you need **two** bounded contexts — payments and orders — sharing the same architectural pattern?

This notebook starts from the working example, restructures it into two bounded contexts, and explores the multi-instance pattern and its current limitations.

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
// ─── Path resolution ───
// The Deno Jupyter kernel's cwd may be notebooks/ or the project root,
// depending on how it's launched. Strip the suffix to normalise.
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";
const EXAMPLE = PROJECT_ROOT + "/notebooks/example-codebase";

// ─── CLI helper ───
// Runs `ksc` as a subprocess, prints combined output, returns exit code.
async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

// ─── Filesystem helpers ───

/** List files in a directory (like the `tree` command). */
async function tree(dir: string): Promise<void> {
  const cmd = new Deno.Command("find", {
    args: [dir, "-type", "f", "-not", "-path", "*/dist/*"],
    stdout: "piped",
  });
  const { stdout } = await cmd.output();
  const files = new TextDecoder().decode(stdout).trim().split("\n")
    .map(f => f.replace(dir + "/", ""))
    .sort();
  console.log(files.join("\n"));
}

/** Copy a directory recursively. Used to clone example-codebase into a temp dir. */
function copyDir(src: string, dest: string): void {
  Deno.mkdirSync(dest, { recursive: true });
  for (const entry of Deno.readDirSync(src)) {
    const s = `${src}/${entry.name}`;
    const d = `${dest}/${entry.name}`;
    if (entry.isDirectory) {
      copyDir(s, d);
    } else {
      Deno.copyFileSync(s, d);
    }
  }
}

console.log("CLI path:", KSC);
console.log("Example codebase:", EXAMPLE);

---

## Part 1: The Example Codebase (Single Instance)

First, let's see the example codebase as-is — a single clean-architecture shop.

In [ ]:
console.log("=== Example codebase structure ===");
await tree(EXAMPLE);

console.log("\n=== architecture.ts ===");
console.log(Deno.readTextFileSync(`${EXAMPLE}/architecture.ts`));

In [ ]:
await ksc("check", EXAMPLE);

5 contracts, all passing. The single `ShopContext` instance maps to `src/` with three layers:
- `src/domain/` — pure entities and value objects (Order, Product, Customer, Money)
- `src/application/` — use cases and port interfaces
- `src/infrastructure/` — adapters implementing ports, HTTP API entry point

Contracts enforced:
- **noDependency**: domain cannot import from application or infrastructure; application cannot import from infrastructure
- **purity**: domain has no Node.js built-in imports
- **mustImplement**: every port interface in application has an adapter in infrastructure

---

## Part 2: Two Bounded Contexts

Now let's scale this out. We want two bounded contexts — **payments** and **orders** — each with its own domain/application/infrastructure layers, sharing the same Kind definition.

We'll copy the example codebase's source files into a new structure.

In [ ]:
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-bounded-ctx-" });
console.log("Working directory:", DEMO);

// Copy the example codebase's domain files into both contexts
// Payments gets the domain entities + its own application/infrastructure
// Orders gets the same structure with order-specific files

// ─── Payments context (reuses domain entities from the example) ───

copyDir(`${EXAMPLE}/src/domain`, `${DEMO}/src/payments/domain`);

Deno.mkdirSync(`${DEMO}/src/payments/application`, { recursive: true });
Deno.writeTextFileSync(`${DEMO}/src/payments/application/payment-gateway.port.ts`, `
export interface PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean;
  refund(transactionId: string): boolean;
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/payments/application/process-payment.ts`, `
import { Money } from '../domain/money';
import { PaymentGatewayPort } from './payment-gateway.port';

export function processPayment(
  customerId: string,
  amount: Money,
  gateway: PaymentGatewayPort,
): boolean {
  return gateway.charge(customerId, Math.round(amount.amount * 100));
}
`.trimStart());

Deno.mkdirSync(`${DEMO}/src/payments/infrastructure`, { recursive: true });
Deno.writeTextFileSync(`${DEMO}/src/payments/infrastructure/stripe-gateway.ts`, `
import { PaymentGatewayPort } from '../application/payment-gateway.port';

export class StripeGateway implements PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean {
    console.log('[Stripe] Charging', customerId, amountCents, 'cents');
    return true;
  }
  refund(transactionId: string): boolean {
    console.log('[Stripe] Refunding', transactionId);
    return true;
  }
}
`.trimStart());

// ─── Orders context (reuses domain entities from the example) ───

copyDir(`${EXAMPLE}/src/domain`, `${DEMO}/src/orders/domain`);

// Copy application layer from example (ports + use cases)
copyDir(`${EXAMPLE}/src/application`, `${DEMO}/src/orders/application`);

// Copy infrastructure adapters from example
Deno.mkdirSync(`${DEMO}/src/orders/infrastructure`, { recursive: true });
Deno.copyFileSync(
  `${EXAMPLE}/src/infrastructure/sql-order-repository.ts`,
  `${DEMO}/src/orders/infrastructure/sql-order-repository.ts`,
);
Deno.copyFileSync(
  `${EXAMPLE}/src/infrastructure/in-memory-catalog.ts`,
  `${DEMO}/src/orders/infrastructure/in-memory-catalog.ts`,
);
Deno.copyFileSync(
  `${EXAMPLE}/src/infrastructure/email-notification.ts`,
  `${DEMO}/src/orders/infrastructure/email-notification.ts`,
);

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

console.log("\n=== Project structure ===");
await tree(DEMO);

### Write `architecture.ts`

We reuse the same Kind definitions from the example (`DomainLayer`, `ApplicationLayer`, `InfrastructureLayer`) but now wrapped in a `BoundedContext` Kind with **two instances** — one for payments, one for orders.

In [ ]:
const architectureTs = `
// ─── Runtime types (inlined for self-contained demo) ───

interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
  mustImplement?: [string, string][];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}

// ─── Kind Definitions (same shape as example-codebase) ───

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface ApplicationLayer extends Kind<"ApplicationLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export interface BoundedContext extends Kind<"BoundedContext"> {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}

// ─── Two instances of the same Kind ───

export const payments = locate<BoundedContext>("src/payments", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const orders = locate<BoundedContext>("src/orders", {
  domain: {},
  application: {},
  infrastructure: {},
});

// ─── Contracts (shared by all BoundedContext instances) ───

export const contracts = defineContracts<BoundedContext>({
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ],
  purity: ["domain"],
  mustImplement: [
    ["application", "infrastructure"],
  ],
});
`.trimStart();

Deno.writeTextFileSync(`${DEMO}/architecture.ts`, architectureTs);
Deno.writeTextFileSync(`${DEMO}/kindscript.json`, JSON.stringify({ definitions: ["architecture.ts"] }, null, 2));

console.log(architectureTs);

---

## Step 1: Check Contracts — Clean State

Both bounded contexts should satisfy all contracts.

In [ ]:
await ksc("check", DEMO);

Both bounded contexts pass. KindScript checked all 5 contracts across both `src/payments/` and `src/orders/`:

- **noDependency**: domain doesn't import from application or infrastructure; application doesn't import from infrastructure
- **purity**: domain has no Node.js built-in imports
- **mustImplement**: every port interface in application has an adapter in infrastructure
- **existence**: all 6 derived directories exist

---

## Step 2: Introduce a Violation in Payments

The payments domain currently has clean `Money`, `Order`, `Product`, and `Customer` entities copied from the example codebase. Let's break purity by adding a Node.js import to `money.ts`.

In [ ]:
// Read the current clean file, then inject a forbidden import
const cleanMoney = Deno.readTextFileSync(`${DEMO}/src/payments/domain/money.ts`);

Deno.writeTextFileSync(`${DEMO}/src/payments/domain/money.ts`,
  `import * as crypto from 'crypto';\n\n` + cleanMoney.replace(
    'Math.random().toString(36)',
    'crypto.randomUUID()'
  )
);

console.log("=== payments/domain/money.ts (first 5 lines) ===");
console.log(Deno.readTextFileSync(`${DEMO}/src/payments/domain/money.ts`).split('\n').slice(0, 5).join('\n'));
console.log("...");
console.log("\nViolation: domain imports 'crypto' (impure)");

In [ ]:
const result = await ksc("check", DEMO);

**BUG: The violation was NOT caught.** KindScript reported all contracts satisfied, even though `payments/domain/money.ts` imports `crypto` — a clear purity violation.

This reveals a limitation in the current implementation: when multiple instances share the same Kind type (`BoundedContext`), the classifier stores instance-to-Kind mappings in a `Map<string, ArchSymbol>` keyed by Kind type name. The second `locate()` call (`orders`) **overwrites** the first (`payments`). Contracts then bind only to the **last** instance.

The payments context is invisible to contract checking. We'll discuss this more in the Issues section.

---

## Step 3: Verify — Orders Violations ARE Caught

To confirm the bug, let's restore payments and instead break the orders context. Since orders is the *last* declared instance, its contracts should be enforced.

In [ ]:
// Restore clean payments domain
Deno.writeTextFileSync(`${DEMO}/src/payments/domain/money.ts`, cleanMoney);

// Break orders domain: import from infrastructure (noDependency violation)
const cleanOrder = Deno.readTextFileSync(`${DEMO}/src/orders/domain/order.ts`);

Deno.writeTextFileSync(`${DEMO}/src/orders/domain/order.ts`,
  `import { SqlOrderRepository } from '../infrastructure/sql-order-repository';\n\n` + cleanOrder
);

console.log("Payments domain: restored (clean)");
console.log("Orders domain:   violation introduced (imports from infrastructure)");

In [ ]:
await ksc("check", DEMO);

**This time the violation is caught** — `KS70001` for the forbidden dependency in `orders/domain/order.ts`. This confirms the bug: only the last instance (`orders`) gets contract enforcement. The first instance (`payments`) is silently skipped.

In [ ]:
// Restore clean state
Deno.writeTextFileSync(`${DEMO}/src/orders/domain/order.ts`, cleanOrder);
await ksc("check", DEMO);

---

## Step 4: Existence Checking — Adding a Third Context

Existence checking (`KS70010`) works correctly across all instances. Let's add a `shipping` context that doesn't have directories yet.

In [ ]:
// Append a third instance to architecture.ts
const archWithShipping = architectureTs + `\nexport const shipping = locate<BoundedContext>("src/shipping", {
  domain: {},
  application: {},
  infrastructure: {},
});
`;

Deno.writeTextFileSync(`${DEMO}/architecture.ts`, archWithShipping);
await ksc("check", DEMO);

`KS70010` fires for all three missing directories under `src/shipping/`. Unlike the contract binding bug, existence checking works correctly across *all* instances.

We can fix this by creating the directories manually.

In [ ]:
// Fix: create the missing directories manually
Deno.mkdirSync(`${DEMO}/src/shipping/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/shipping/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/shipping/infrastructure`, { recursive: true });

console.log("--- After creating directories ---");
await ksc("check", DEMO);

---

## Step 5: Workaround — Separate Kind Types

The contract binding bug only affects shared Kind types. If each bounded context has its own Kind type, contracts bind correctly.

In [ ]:
// Rewrite architecture.ts with separate Kind types per context
Deno.writeTextFileSync(`${DEMO}/architecture.ts`, `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
  mustImplement?: [string, string][];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}

// ─── Payments Context (its own Kind type) ───

export interface PaymentsDomain extends Kind<"PaymentsDomain"> {}
export interface PaymentsApplication extends Kind<"PaymentsApplication"> {}
export interface PaymentsInfrastructure extends Kind<"PaymentsInfrastructure"> {}

export interface PaymentsContext extends Kind<"PaymentsContext"> {
  domain: PaymentsDomain;
  application: PaymentsApplication;
  infrastructure: PaymentsInfrastructure;
}

export const payments = locate<PaymentsContext>("src/payments", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const paymentsContracts = defineContracts<PaymentsContext>({
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ],
  purity: ["domain"],
  mustImplement: [["application", "infrastructure"]],
});

// ─── Orders Context (its own Kind type) ───

export interface OrdersDomain extends Kind<"OrdersDomain"> {}
export interface OrdersApplication extends Kind<"OrdersApplication"> {}
export interface OrdersInfrastructure extends Kind<"OrdersInfrastructure"> {}

export interface OrdersContext extends Kind<"OrdersContext"> {
  domain: OrdersDomain;
  application: OrdersApplication;
  infrastructure: OrdersInfrastructure;
}

export const orders = locate<OrdersContext>("src/orders", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const ordersContracts = defineContracts<OrdersContext>({
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ],
  purity: ["domain"],
  mustImplement: [["application", "infrastructure"]],
});
`.trimStart());

console.log("Rewrote architecture.ts with separate Kind types per context");
await ksc("check", DEMO);

In [ ]:
// Now break payments — this time it SHOULD be caught
Deno.writeTextFileSync(`${DEMO}/src/payments/domain/money.ts`,
  `import * as crypto from 'crypto';\n\n` + cleanMoney
);

console.log("Purity violation in payments/domain/money.ts:");
await ksc("check", DEMO);

With separate Kind types, the payments purity violation is caught (`KS70003`). Each context has its own Kind type so there's no map-key collision in the classifier.

The trade-off: more boilerplate (duplicate interface definitions), but correct contract enforcement.

---

## Issues and Design Discussion

### 1. Contract-to-Instance Binding Bug

**Demonstrated above.** When two instances share the same Kind type, contracts only check the *last* instance declared.

**Root cause:** In `classify-ast.service.ts`, the classifier stores instance-to-Kind mappings in a `Map<string, ArchSymbol>` keyed by Kind type name (line 96: `instanceSymbols.set(kindName, result.symbol)`). Since Map keys are unique, the second `locate<BoundedContext>()` overwrites the first.

**Impact:** In the multi-instance pattern — the primary use case for shared Kind types — only the last declared context gets contract enforcement. All earlier contexts are silently unchecked.

**Workaround:** Use separate Kind types per context (shown in Step 5).

**Fix:** Change `instanceSymbols` from `Map<string, ArchSymbol>` to `Map<string, ArchSymbol[]>` and fan out contract binding to all instances of a given Kind type.

### 2. Shared vs Per-Instance Contracts

With the shared-Kind approach, one `defineContracts<BoundedContext>()` applies to all instances. This is often desirable — all clean architecture contexts should enforce the same rules.

If you need **different rules per context** (e.g., payments has stricter purity), you must use separate Kind types. This is a fundamental design choice: same Kind type = same contracts.

### 3. Cross-Context Dependencies

KindScript's `noDependency` contracts operate *within* a bounded context (e.g., payments.domain cannot import from payments.infrastructure). But what about **cross-context** dependencies (payments importing from orders)?

This isn't currently expressible. You'd need a higher-level Kind that has the bounded contexts as members and defines inter-context dependency rules.

In [ ]:
Deno.removeSync(DEMO, { recursive: true });
console.log("Working directory cleaned up.");

---

## Summary

| Pattern | Status |
|---------|--------|
| Single instance (example-codebase) | Works — all 5 contracts enforced |
| Multi-instance, shared Kind type | **BUG** — only last instance checked |
| Multi-instance, separate Kind types | Works — correct enforcement (more boilerplate) |
| Existence checking | Works — across all instances |

**Key takeaway:** For now, use separate Kind types per bounded context to get correct contract enforcement. The shared-Kind multi-instance pattern is the intended design but has a contract binding bug (`classify-ast.service.ts` line 96) that needs to be fixed.